In [26]:
!pip install --upgrade torch


# Imports

In [27]:
import torch
import argparse
from torch.optim.optimizer import Optimizer, required
import torch
import torchvision
import torchvision.transforms as transforms
import random
import numpy as np
import json
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, SequentialLR, LinearLR
import matplotlib.pyplot as plt
import time
from torch.amp import GradScaler, autocast
import os
from google.colab import drive

In [28]:
drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    raise RuntimeError("Google Drive not mounted correctly!")

Mounted at /content/drive


## Costants

In [29]:
BATCH_SIZE = 64

## Model

In [30]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5)

        # Layer fully connected
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x):
        # Layer convolutivi con ReLU e max-pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        # Flatten per i layer fully connected
        x = torch.flatten(x, 1)

        # Layer fully connected con ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)

        return x

## Early stopping

In [31]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, delta=0, path='/content/drive/MyDrive/Early2checkpoint.pt', verbose=False):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path
        self.verbose = verbose

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, optimizer=None):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer else None,
            'val_loss_min': val_loss
        }
        torch.save(checkpoint, self.path)
        self.val_loss_min = val_loss



## Training function

In [32]:
def train_model_with_hyperparams(train_loader, val_loader, test_loader, num_epochs, device, checkpoint_path, optimizer, patience=5):
    model = LeNet5().to(device)

    warmup_scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=5)
    scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, CosineAnnealingLR(optimizer, T_max=num_epochs - 5)], milestones=[5])
    #scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    # Checkpoint recovery
    start_epoch = 0
    if os.path.exists(checkpoint_path):
        print(f"Checkpoint found at {checkpoint_path}. Loading model state...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch']
        print(f"Resuming training from epoch {start_epoch + 1}")
    else:
        print(f"No checkpoint found at {checkpoint_path}. Training a new model from scratch.")
        checkpoint = None

    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(start_epoch, num_epochs):
        model.train()
        train_loss_total, train_correct, train_total = 0, 0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss_total += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss = train_loss_total / len(train_loader)
        train_acc = 100. * train_correct / train_total
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # Validation loop
        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss_total += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_loss = val_loss_total / len(val_loader)
        val_acc = 100. * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # Update scheduler after validation
        scheduler.step()

        early_stopping(val_loss, model)

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracies': train_accuracies,
            'val_accuracies': val_accuracies
        }

        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint saved to {checkpoint_path} at epoch {epoch + 1}")

        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    # Load the best model
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Test loop
    model.eval()
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(inputs)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_acc = 100. * test_correct / test_total
    print(f"Final Test Accuracy: {test_acc:.2f}%")
    return train_losses, val_losses, train_accuracies, val_accuracies, test_acc


## Defining different optimizers

In [33]:

class LARS(Optimizer):
    def __init__(self, params, lr=0.1, momentum=0.9, weight_decay=0.0, eta=0.001, eps=1e-8):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if momentum < 0.0 or momentum >= 1.0:
            raise ValueError(f"Invalid momentum: {momentum}")
        if weight_decay < 0.0:
            raise ValueError(f"Invalid weight_decay: {weight_decay}")
        if eta <= 0.0:
            raise ValueError(f"Invalid eta: {eta}")

        defaults = dict(lr=lr, momentum=momentum, weight_decay=weight_decay, eta=eta, eps=eps)
        super(LARS, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']
            weight_decay = group['weight_decay']
            eta = group['eta']
            eps = group['eps']

            for param in group['params']:
                if param.grad is None:
                    continue

                grad = param.grad.data
                if grad.isinf().any() or grad.isnan().any():
                    print(f"Skipping update for parameter {param} due to NaN/Inf gradients.")
                    continue

                if weight_decay != 0:
                    grad.add_(param.data, alpha=weight_decay)

                param_norm = param.data.norm(2).clamp(min=eps)
                grad_norm = grad.norm(2).clamp(min=eps)
                local_lr = eta * param_norm / grad_norm

                if 'momentum_buffer' not in self.state[param]:
                    buf = self.state[param]['momentum_buffer'] = torch.clone(grad).detach()
                else:
                    buf = self.state[param]['momentum_buffer']
                    buf.mul_(momentum).add_(grad)

                param.data.add_(buf, alpha=-local_lr * lr)

        return loss


class LAMB(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.01):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Invalid beta parameter at index 0: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Invalid beta parameter at index 1: {betas[1]}")
        if weight_decay < 0.0:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super(LAMB, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            betas = group['betas']
            eps = group['eps']
            weight_decay = group['weight_decay']

            for param in group['params']:
                if param.grad is None:
                    continue

                grad = param.grad.data
                if grad.isinf().any() or grad.isnan().any():
                    print(f"Skipping update for parameter {param} due to NaN/Inf gradients.")
                    continue

                state = self.state[param]
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(param.data)
                    state['exp_avg_sq'] = torch.zeros_like(param.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = betas
                state['step'] += 1

                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                denom = exp_avg_sq.sqrt().add_(eps)
                r1 = param.data.norm(2).clamp(min=eps)
                r2 = exp_avg.norm(2).clamp(min=eps)
                trust_ratio = r1 / r2 if r2 != 0 else 1.0

                step_size = lr * trust_ratio
                if weight_decay != 0:
                    param.data.add_(param.data, alpha=-weight_decay)

                param.data.addcdiv_(exp_avg, denom, value=-step_size)

        return loss


class AdamW(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Invalid beta parameter at index 0: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Invalid beta parameter at index 1: {betas[1]}")
        if weight_decay < 0.0:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step."""
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for param in group['params']:
                if param.grad is None:
                    continue

                grad = param.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients')

                if grad.isinf().any() or grad.isnan().any():
                    print(f"Inf or NaN gradients found for {param}")
                    continue

                state = self.state[param]

                # initial state
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(param.data)
                    state['exp_avg_sq'] = torch.zeros_like(param.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # update moving averages
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                denom = (exp_avg_sq.sqrt() / (1 - beta2 ** state['step'])).add_(group['eps'])
                step_size = group['lr'] / (1 - beta1 ** state['step'])

                # apply weight decay
                if group['weight_decay'] != 0:
                    param.data.add_(param.data, alpha=-group['weight_decay'] * group['lr'])

                # update params
                param.data.addcdiv_(exp_avg, denom, value=-step_size)

        return loss


class SDGM(Optimizer):
    def __init__(self, params, lr=1e-3, momentum=0.9, weight_decay=0.0, eps=1e-8):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if momentum < 0.0 or momentum >= 1.0:
            raise ValueError(f"Invalid momentum: {momentum}")
        if weight_decay < 0.0:
            raise ValueError(f"Invalid weight_decay: {weight_decay}")

        defaults = dict(lr=lr, momentum=momentum, weight_decay=weight_decay, eps=eps)
        super(SDGM, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step."""
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']
            weight_decay = group['weight_decay']
            eps = group['eps']

            for param in group['params']:
                if param.grad is None:
                    continue


                grad = param.grad.data

                if grad.isinf().any() or grad.isnan().any():
                    print(f"Inf or NaN gradients found for {param}")
                    continue


                if weight_decay != 0:
                    grad.add_(param.data, alpha=weight_decay)

                # Dual gradient updates
                param_norm = param.data.norm(2)
                grad_norm = grad.norm(2)
                dual_lr = lr * param_norm / (grad_norm + eps)

                # apply momentum updates
                if 'momentum_buffer' not in self.state[param]:
                    buf = self.state[param]['momentum_buffer'] = torch.clone(grad).detach()
                else:
                    buf = self.state[param]['momentum_buffer']
                    buf.mul_(momentum).add_(grad)

                param.data.add_(buf, alpha=-dual_lr)

        return loss


In [34]:
def parse_args():
    parser = argparse.ArgumentParser(description='Large-Batch Optimizer Selection')
    parser.add_argument('--optimizer', type=str, default='SGDM', choices=['SGDM', 'AdamW', 'LARS', 'LAMB'],
                        help='Select optimizer (SGDM, AdamW, LARS, LAMB)')
    #parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    #parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
    args = parser.parse_args()
    return args


## Function to get train, test and val dataset

In [35]:
def get_dataset(batch_size=100):

    # Define the transform to only convert the images to tensors (without normalization yet)
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Load the CIFAR-100 training dataset
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=False, num_workers=2)

    # Initialize sums for calculating mean and std
    mean = torch.zeros(3)
    std = torch.zeros(3)

    for images, _ in train_loader:
        # Compute mean and std for each channel
        mean += images.mean(dim=[0, 2, 3])
        std += images.std(dim=[0, 2, 3])

    mean /= len(train_loader)
    std /= len(train_loader)

    print("Mean: ", mean)
    print("Std: ", std)

    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean[0].item(), mean[1].item(), mean[2].item()],
                            std=[std[0].item(), std[1].item(), std[2].item()])
    ])

    transform_test = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean[0].item(), mean[1].item(), mean[2].item()],
                            std=[std[0].item(), std[1].item(), std[2].item()])
    ])

    # Load CIFAR-100 dataset
    start_time = time.time()
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=True,
        download=True,
        transform=transform_train
    )
    test_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=False,
        download=True,
        transform=transform_test
    )
    print(f"Dataset loading time: {time.time() - start_time:.2f} seconds")

    # Split training and validation sets
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # Data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # Debugging: Check DataLoader outputs
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"Batch {i}: inputs shape: {inputs.shape}, labels shape: {labels.shape}")
        if i == 10:  # Test first 10 batches
            break
    print(f"Data loading for 10 batches completed.")

    return train_loader, val_loader, test_loader

## Main

In [36]:
if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LeNet5().to(device)

    #args = parse_args()

    class Args:
        def __init__(self):
            self.optimizer = 'SDGM'  # Choose from: 'SGDM', 'AdamW', 'LARS', 'LAMB'
            #self.batch_size = 32

    args = Args() # replace "args = parse_args()" with this line


    all_train_losses = {}
    all_val_losses = {}
    all_train_accuracies = {}
    all_val_accuracies = {}
    all_test_accuracies = {}

    batch_sizes = [64, 128, 256, 512, 1024]

    for batch_size in batch_sizes:
        train_loader, val_loader, test_loader = get_dataset(batch_size)

        if args.optimizer == "LARS":
            print("using LARS optimizer...")
            optimizer = LARS(model.parameters(), lr=0.1 * (batch_size / 256), momentum=0.9, weight_decay=0.01)
        elif args.optimizer == "AdamW":
            print("using AdamW optimizer...")
            optimizer = AdamW(model.parameters(), lr=0.001 * (batch_size / 64), weight_decay=0.01)
        elif args.optimizer == "LAMB":
            print("using LAMB optimizer...")
            optimizer = LAMB(model.parameters(), lr=0.001 * (batch_size / 64), weight_decay=0.01)
        elif args.optimizer == "SDGM":
            print("using SDGM optimizer...")
            optimizer = SDGM(model.parameters(), lr=0.01 * (batch_size / 64), weight_decay=0.005)
        else:
            raise ValueError("Invalid optimizer name")

        train_losses, val_losses, train_accuracies, val_accuracies, test_acc = train_model_with_hyperparams(
            train_loader,
            val_loader,
            test_loader,
            50,  # epoch
            device,
            "/content/drive/MyDrive/checkpoint1.pth",
            optimizer,
            patience=5
        )

        # Salvataggio dei risultati
        all_train_losses[batch_size] = train_losses
        all_val_losses[batch_size] = val_losses
        all_train_accuracies[batch_size] = train_accuracies
        all_val_accuracies[batch_size] = val_accuracies
        all_test_accuracies[batch_size] = test_acc

    plt.figure(figsize=(14, 10))

    plt.subplot(2, 1, 1)
    for batch_size in batch_sizes:
        plt.plot(range(1, len(all_train_losses[batch_size]) + 1), all_train_losses[batch_size], label=f'Train Loss (BS={batch_size})')
        plt.plot(range(1, len(all_val_losses[batch_size]) + 1), all_val_losses[batch_size], '--', label=f'Val Loss (BS={batch_size})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss for Different Batch Sizes')
    plt.legend()

    plt.subplot(2, 1, 2)
    for batch_size in batch_sizes:
        plt.plot(range(1, len(all_train_accuracies[batch_size]) + 1), all_train_accuracies[batch_size], label=f'Train Acc (BS={batch_size})')
        plt.plot(range(1, len(all_val_accuracies[batch_size]) + 1), all_val_accuracies[batch_size], '--', label=f'Val Acc (BS={batch_size})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy for Different Batch Sizes')
    plt.legend()

    plt.tight_layout()

    save_path = '/content/drive/My Drive/Colab Notebooks/Traning_summary/Batch_Size_Comparison'
    plt.savefig(save_path + '_results.png')
    print("Batch size comparison results saved to Google Drive as 'Batch_Size_Comparison_results.png'")
    plt.show()

    summary_path = save_path + '_summary.txt'
    with open(summary_path, 'w') as f:
        for batch_size in batch_sizes:
            f.write(f"Batch Size: {batch_size}\n")
            f.write(f"Final Test Accuracy: {all_test_accuracies[batch_size]:.2f}%\n")
            f.write(f"Train Losses: {all_train_losses[batch_size]}\n")
            f.write(f"Validation Losses: {all_val_losses[batch_size]}\n")
            f.write(f"Train Accuracies: {all_train_accuracies[batch_size]}\n")
            f.write(f"Validation Accuracies: {all_val_accuracies[batch_size]}\n")
            f.write("\n")
    print(f"Training summaries saved to Google Drive as 'Batch_Size_Comparison_summary.txt'")

Files already downloaded and verified
Mean:  tensor([0.5071, 0.4865, 0.4409])
Std:  tensor([0.2667, 0.2558, 0.2754])
Files already downloaded and verified
Files already downloaded and verified
Dataset loading time: 2.01 seconds
Batch 0: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 1: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 2: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 3: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 4: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 5: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 6: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 7: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 8: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 9: inputs shape: to

KeyboardInterrupt: 